In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import seaborn as sns   
import matplotlib.pyplot as plt

train_features = pd.read_csv("train_new_features.csv").sort_values(by='pid')
test = pd.read_csv("test_new_features.csv").sort_values(by='pid')
train_labels = pd.read_csv("train_labels.csv").sort_values(by='pid')


In [ ]:
train_labels.head()

In [ ]:
train_features.head(13)

In [ ]:
VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

In [ ]:
# Let's now divide our training set into train and validation set

"""
Splitting the dataset into train 80% and  val 20%  
"""
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=2020)


In [ ]:
X_train

In [ ]:
y_train

In [ ]:
feature_cols = X_train.columns.values[(X_train.columns.values != 'pid') & (X_train.columns.values != 'Time')]

X_train = X_train[feature_cols]
X_val = X_val[feature_cols]


In [ ]:
# Regression models for comparison
'''
models is a list containing the objects for ML algorithms.
results is a dictionary where key is algorithm name and value is MAE.
We train for each model and predict on Val.
We use the ground truth and predictions to calculate MAE.
The MAE's for each model are stored in results.
'''
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import pearsonr
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor

ml_models = [LinearRegression(),
          GradientBoostingRegressor(random_state = 0), 
          KNeighborsRegressor(),
          RandomForestRegressor(random_state = 0),
          SVR()]



VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']


def regression(label):
    
    results = {}
    for model in ml_models:

        # Instantiate and fit Regressor Model
        reg_model = model
        reg_model.fit(X_train, y_train[label])

        # Make predictions with model
        y_test_preds = reg_model.predict(X_val)

        # Grab model name and store results associated with model
        name = str(model).split("(")[0]

        results[name] = r2_score(y_val[label], y_test_preds)
        print('{} done.'.format(name))


    print('R2 score')
    for i in results.keys():
        print('{} : {}'.format(i,results[i]))

    # Plot barplot for MAE

    ind = range(len(results))
    max_r2 = max(results.values())

    fig = plt.figure(figsize=(15,6))
    ax = fig.subplots()
    ax.bar(ind, list(results.values()), align='center', 
            color = 'lightblue', alpha=0.8)
    plt.xticks(ind, results.keys())
    ax.axhline(y=max_r2, xmin=0, xmax=100, ls='-', color='red')
    plt.show()

In [ ]:
label = VITALS[0]
print (label)
regression(label)



'''
LABEL_RRate
LinearRegression done.
GradientBoostingRegressor done.
KNeighborsRegressor done.
RandomForestRegressor done.
SVR done.
R2 score
LinearRegression : 0.346292396049945
GradientBoostingRegressor : 0.40853105319770444
KNeighborsRegressor : -0.04090803120945519
RandomForestRegressor : 0.3979781097577445
SVR : 0.18327352032804545

'''

In [ ]:
label = VITALS[1]
print ("##########",label,"###########")
regression(label)



'''
########## LABEL_ABPm ###########
LinearRegression done.
GradientBoostingRegressor done.
KNeighborsRegressor done.
RandomForestRegressor done.
SVR done.
R2 score
LinearRegression : 0.592904590103991
GradientBoostingRegressor : 0.6079879627877394
KNeighborsRegressor : 0.4370851907863198
RandomForestRegressor : 0.60514847180928
SVR : 0.5391456017012313

'''

In [ ]:
label = VITALS[2]
print (label)
regression(label)


'''
LABEL_SpO2
LinearRegression done.
GradientBoostingRegressor done.
KNeighborsRegressor done.
RandomForestRegressor done.
SVR done.
R2 score
LinearRegression : 0.45433326748317515
GradientBoostingRegressor : 0.3981090074027216
KNeighborsRegressor : -0.06081644405509867
RandomForestRegressor : 0.37723587380543366
SVR : 0.11255840204357526

'''

In [ ]:
label = VITALS[3]
print (label)
regression(label)


'''
LABEL_Heartrate
LinearRegression done.
GradientBoostingRegressor done.
KNeighborsRegressor done.
RandomForestRegressor done.
SVR done.
R2 score
LinearRegression : 0.593564254367609
GradientBoostingRegressor : 0.624227589085022
KNeighborsRegressor : 0.3943120087391996
RandomForestRegressor : 0.6147944170621644
SVR : 0.5295699779291231


'''

In [ ]:
# 3. Ridge regression using all features


from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


alpha = [0, 1000, 10000, 40000, 90000, 100000]


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)


for label in VITALS:
    
    print ("Target = ", label)
    y_temp = y_train[label]
    for i in alpha:
        model = Ridge(alpha = i, fit_intercept = False)
        print("Alpha = ", i, "R2 score = ", np.mean(cross_val_score(model, X_train, y_temp, cv=10, scoring = 'r2')))
    

    
'''

Target =  LABEL_RRate
Alpha =  0.1 R2 score =  -10.86111657439984
Alpha =  1 R2 score =  -11.98977482355494
Alpha =  10 R2 score =  -8.622023360577263
Alpha =  1000 R2 score =  0.3833156346672817
Alpha =  10000 R2 score =  0.3885456610947021
Alpha =  40000 R2 score =  0.38892500213972137
Alpha =  90000 R2 score =  0.3858134922593838
Alpha =  100000 R2 score =  0.38499156970780685
Target =  LABEL_ABPm
Alpha =  0.1 R2 score =  -55.96038837039049
Alpha =  1 R2 score =  -56.90985564619338
Alpha =  10 R2 score =  -55.984600720901724
Alpha =  1000 R2 score =  -3.053959115227792
Alpha =  10000 R2 score =  0.5031267093430214
Alpha =  40000 R2 score =  0.5838715357512532
Alpha =  90000 R2 score =  0.588713763227594
Alpha =  100000 R2 score =  0.5888388017361061
Target =  LABEL_SpO2
Alpha =  0.1 R2 score =  -138.938104289817
Alpha =  1 R2 score =  -173.26384941090583
Alpha =  10 R2 score =  -172.57771289828378
Alpha =  1000 R2 score =  -9.847322628863363
Alpha =  10000 R2 score =  0.095101795462909
Alpha =  40000 R2 score =  0.2745952852313543
Alpha =  90000 R2 score =  0.24866806922253915
Alpha =  100000 R2 score =  0.2419201429396946
Target =  LABEL_Heartrate
Alpha =  0.1 R2 score =  0.3139765976527324
Alpha =  1 R2 score =  0.17145912150564943
Alpha =  10 R2 score =  0.13128863914438416
Alpha =  1000 R2 score =  0.5829783084089712
Alpha =  10000 R2 score =  0.6223412979739278
Alpha =  40000 R2 score =  0.6218643531121091
Alpha =  90000 R2 score =  0.6210104852686391
Alpha =  100000 R2 score =  0.620861296704975

    
'''

In [ ]:
# Let's tune our Random Forest regressor for hyperparameters

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score


VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']


for label in VITALS:
    
    print(label)
    
    reg = RandomForestRegressor(random_state=42)
    
    param_grid = { 
    'n_estimators': [1000]}    
    
    CV_rf = GridSearchCV(estimator=reg, param_grid=param_grid, cv = 5, scoring = 'r2', n_jobs = -1)
    
    CV_rf.fit(X_train, y_train[label])
    
    y_pred = CV_rf.predict(X_val)
    
    print("Best parameters after Grid Search CV")
        
    print(CV_rf.best_params_)                  
        
    R2_score = r2_score(y_val[label], y_pred)
    
    print("R2 score with best parameters = ", R2_score)
    
    print("----------------------------------------------")
    
    
'''
LABEL_RRate
Best parameters after Grid Search CV
{'n_estimators': 400}
R2 score with best parameters =  0.40483515650529134
----------------------------------------------
LABEL_ABPm
Best parameters after Grid Search CV
{'n_estimators': 400}
R2 score with best parameters =  0.6073676454810253
----------------------------------------------
LABEL_SpO2
Best parameters after Grid Search CV
{'n_estimators': 400}
R2 score with best parameters =  0.3896437001125159
----------------------------------------------
LABEL_Heartrate
Best parameters after Grid Search CV
{'n_estimators': 400}
R2 score with best parameters =  0.6191019592726559
----------------------------------------------

'''

In [ ]:
# Let's see if n_estimators = 200 is good enough for us

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression


VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']


for label in VITALS:
    
    print(label)
    
    if label == 'LABEL_SpO2':
        lreg = LinearRegression().fit(X_train, y_train[label])
        y_pred = lreg.predict(X_val)
        R2_score = r2_score(y_val[label], y_pred)
        print("R2 score = ", R2_score)
    
        print("----------------------------------------------")
        
        
    else:
        
        reg = RandomForestRegressor(n_estimators = 200, random_state=42)

        reg.fit(X_train, y_train[label])

        y_pred = reg.predict(X_val)             

        R2_score = r2_score(y_val[label], y_pred)

        print("R2 score = ", R2_score)

        print("----------------------------------------------")

In [ ]:
## Conclusions 

## There is hardly any change in R2 score when n_estimators are changed from 400 to 200. So we will go with 200 estimators. 

In [ ]:
# 1. Simple Linear regression using all features
'''
from sklearn.linear_model import LinearRegression

for label in VITALS:
    print ("Target = ", label)
    y_temp = y_train[label]
    reg = LinearRegression().fit(X_train, y_temp) # fitting the data
    print("training score = ", reg.score(X_train, y_temp))  # How well do we fit


Target =  LABEL_RRate
training score =  0.4101943384913194
Target =  LABEL_ABPm
training score =  0.6030440190107336
Target =  LABEL_SpO2
training score =  0.37633609527438916
Target =  LABEL_Heartrate
training score =  0.6303946090937576
'''

# Score on validation set 



from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

for label in VITALS:
    print ("Target = ", label)
    y_temp = y_train[label]
    reg = LinearRegression().fit(X_train, y_temp) # fitting the data
    y_pred_val = reg.predict(X_val)  
    print("R2 score = ", r2_score(y_val[label], y_pred_val))
  
'''
Target =  LABEL_RRate
R2 score =  0.36043130617783037
Target =  LABEL_ABPm
R2 score =  0.5981269926232222
Target =  LABEL_SpO2
R2 score =  0.4629634518153568
Target =  LABEL_Heartrate
R2 score =  0.6023979649232624


'''

In [ ]:
# 3. Ridge regression using only one feature


'''
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

X_train_temp = X_train['RRate']
X_train_temp = X_train_temp[:, np.newaxis]
y_train_temp = y_train['LABEL_RRate']

alpha = [1, 10, 100, 200, 300, 500, 1000, 2000, 10000]

for label in VITALS:
    print ("Target = ", label)
    y_temp = y_train[label]
    for i in alpha:
        print ("Alpha = ",i)
        model = Ridge(alpha = i, fit_intercept = True)
        print(np.mean(cross_val_score(model, X_train_temp, y_train_temp, cv=5, scoring = 'r2')))
    break
    
Target =  LABEL_RRate
Alpha =  1
0.3730519805372717
Alpha =  10
0.37305199140415507
Alpha =  100
0.373052009980947
Alpha =  200
0.3730518389036903
Alpha =  300
0.3730514666259451
Alpha =  500
0.3730501209755694
Alpha =  1000
0.37304327950197713
Alpha =  2000
0.3730149544282296
Alpha =  10000
0.3721505170640766
'''